# End-to-end NLP: News Headline classifier

### Setup execution role and session

Let's start by specifying:

- The S3 bucket and prefix that you want to use for training and model data. This should be within the same region as the Notebook Instance, training, and hosting. If you don't specify a bucket, SageMaker SDK will create a default bucket following a pre-defined naming convention in the same region. 
- The IAM role ARN used to give SageMaker access to your data. It can be fetched using the **get_execution_role** method from sagemaker python SDK.

In [ ]:
%%time
import sagemaker
from sagemaker import get_execution_role

role = get_execution_role()
print(role)
sess = sagemaker.Session()

### Download News Aggregator Dataset available at the public UCI dataset repository (these files should already be downloaded in previous notebook)

In [ ]:
import src.preprocessing

In [ ]:
src.preprocessing.download_dataset()

### Let's visualize the dataset

We will load the newsCorpora.csv file to a Pandas dataframe for our data processing work

In [ ]:
import pandas as pd
import re
import numpy as np
import os

In [ ]:
column_names = ["TITLE", "URL", "PUBLISHER", "CATEGORY", "STORY", "HOSTNAME", "TIMESTAMP"]
df = pd.read_csv('newsCorpora.csv', names=column_names, header=None, delimiter='\t')
df.head()

#### For this exercice we'll only use the title (Headline) of the news story and the category as our target variable

In [ ]:
from collections import Counter
Counter(df['CATEGORY'])

The dataset has four categories: Business (b), Science & Technology (t), Entertainment (e) and Health & Medicine (m).

## Natural Language pre processing

We will do some basic processing of the text data to convert it into numerical form that the algorithm will be able to consume to create a model.
We will do typical pre processing for NLP workloads such as: dummy encoding the labels, tokenizing the documents and set fixed sequence lengths for input feature dimension, padding documents to have fixed length input vectors.

#### Dummy encode the labels

In [ ]:
encoded_y=src.preprocessing.dummy_encode_labels(df,'CATEGORY')

In [ ]:
df['CATEGORY'][1]

In [ ]:
encoded_y[0]

#### Tokenize documents and set fixed sequence lengths for input feature dimension.

In [ ]:
padded_docs, tokenizer=src.preprocessing.tokenize_pad_docs(df,'TITLE')

In [ ]:
df['TITLE'][1]

In [ ]:
padded_docs[0]

### Import word embeddings

The vectors.txt file is the output of the blazingtext_word2vec_text8.ipynb notebook. This will have a list of vector representations for each word in our vocabulary.

In [ ]:
embedding_matrix=src.preprocessing.get_word_embeddings(tokenizer)

In [ ]:
mkdir ./data/ ./data/embeddings/

### Save embedding matrix to push to S3 for Sagemaker to use during training

In [ ]:
#embedding_matrix.dump("ingredients-embedding-matrix.dat")
np.save(file="./data/embeddings/docs-embedding-matrix",
        arr=embedding_matrix,
        allow_pickle=False)
vocab_size=embedding_matrix.shape[0]
print(embedding_matrix.shape)

### Train, test split

In this section we will prep the data for ingestion for the algortihm. Split the data set in train and test samples and uplad the data to S3

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(padded_docs, encoded_y, test_size=0.2, random_state=42)

In [ ]:
!mkdir data/train/ data/test/

In [ ]:
np.save('./data/train/train_X.npy', X_train)
np.save('./data/train/train_Y.npy', y_train)
np.save('./data/test/test_X.npy', X_test)
np.save('./data/test/test_Y.npy', y_test)

In [ ]:
s3_bucket = sess.default_bucket()
s3_prefix = 'news'

In [ ]:
traindata_s3_prefix = '{}/data/train'.format(s3_prefix)
testdata_s3_prefix = '{}/data/test'.format(s3_prefix)
embeddings_s3_prefix='{}/data/embeddings'.format(s3_prefix)
output_s3 = 's3://{}/{}/models/'.format(s3_bucket, s3_prefix)
code_location_s3 = 's3://{}/{}/codes'.format(s3_bucket, s3_prefix)

In [ ]:
train_s3 = sess.upload_data(path='./data/train/', bucket=s3_bucket, key_prefix=traindata_s3_prefix)
test_s3 = sess.upload_data(path='./data/test/', bucket=s3_bucket, key_prefix=testdata_s3_prefix)
embeddings_s3 = sess.upload_data(path='./data/embeddings/', bucket=s3_bucket, key_prefix=embeddings_s3_prefix)


In [ ]:
inputs = {'train':train_s3, 'test': test_s3, 'embeddings': embeddings_s3}

print(inputs)

## Sagemaker training with differentiated infrastructure

We will use the high level TensorFlow SDK to train our keras model using Sagemaker. We have packaged the code we used to build and train our model in the previous notebook (headline-classifier-local.ipynb). The training script is available in the src directory. You can read the details of the script and format in the python file: keras_script.py.

In [ ]:
import sagemaker
from sagemaker.tensorflow import TensorFlow

### Define hyperparameters to push to algorithm

In [ ]:
hyperparameters = {'epochs': 5, 'vocab_size':vocab_size, 'num_classes':4}

We have our `TensorFlow` estimator object, we have set the hyper-parameters for this object and we have our data channels linked with the algorithm. The only  remaining thing to do is to train the algorithm. The following command will train the algorithm. Training the algorithm involves a few steps. Firstly, the instance that we requested while creating the `TensorFlow` estimator classes is provisioned and is setup with the appropriate libraries. Then, the data from our channels are downloaded into the instance. Once this is done, the training job begins. The provisioning and data downloading will take some time, depending on the size of the data. 

Once the job has finished a "Job complete" message will be printed. The trained model can be found in the S3 bucket that was setup as `output_path` in the estimator.

We will run the training job using a ml.p2.xlarge instance (GPUs) to accelerate our training. If your account runs into resource limits please use a ml.c5.xlarge instance. 

In [ ]:
tf_estimator = TensorFlow(entry_point='keras_script.py',
                       source_dir='./src',
                        role=role,
                        train_instance_count=1, 
                        train_instance_type='ml.p3.2xlarge',
                        framework_version='1.14', py_version='py2',
                        distributions={'parameter_server': {'enabled': True}},
                        hyperparameters=hyperparameters,
                         metric_definitions=[
                           {'Name': 'validation:loss', 'Regex': 'loss: ([0-9\\.]+)'},
                           {'Name': 'validation:accuracy', 'Regex': 'acc: ([0-9\\.]+)'}]
                        )
tf_estimator.fit(inputs)

While the training job is running take a minute to look at the keras_script.py script. You can see how we have adapted the our original local code from headline-classifier-local.ipynb to run on Sagemaker.

## Hosting / Inference
Once the training is done, we can deploy the trained model as an Amazon SageMaker real-time hosted endpoint. This will allow us to make predictions (or inference) from the model. Note that we don't have to host on the same type of instance that we used to train. Because instance endpoints will be up and running for long, it's advisable to choose a cheaper instance for inference.


In [ ]:
predictor = tf_estimator.deploy(initial_instance_count=1,
                           instance_type='ml.t2.medium')

### Your model should now be in production as a RESTful API!

Let's evaluate our model with an example headline. You can be creative with your headlines.

In [ ]:
from keras.preprocessing.sequence import pad_sequences
example_doc=['The markets were bullish after news of the merger']
# integer encode the document
encoded_example = tokenizer.texts_to_sequences(example_doc)

# pad documents to a max length of 4 words
max_length = 40
padded_example = pad_sequences(encoded_example, maxlen=max_length, padding='post')

The result represents the probability or confidence of the model that the headline represents one of the classes: 'b', 'e', 'm', 't'

In [ ]:
predictor.predict(padded_example.tolist())

## Clean up

In [ ]:
sess.delete_endpoint(predictor.endpoint)

## HPO - Automatic Hyperparameter tuning (Optional)

In [ ]:
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner

In [ ]:
#hyperparameter_ranges = {'learning_rate': ContinuousParameter(0.01, 0.2)}
hyperparameter_ranges = {'epochs': IntegerParameter(2, 7)}

In [ ]:
objective_metric_name = 'loss'
objective_type = 'Minimize'
metric_definitions = [{'Name': 'loss',
                       'Regex': 'loss = ([0-9\\.]+)'}]

In [ ]:
hyperparameters = {'epochs': 5, 'vocab_size':vocab_size, 'num_classes':4}

In [ ]:
tf_estimator = TensorFlow(entry_point='keras_script.py',
                       source_dir='./src',
                        role=role,
                        train_instance_type='ml.c5.xlarge',
                        train_instance_count=1,
                        framework_version='1.14', 
                        py_version='py2',
                        hyperparameters=hyperparameters)

In [ ]:
tuner = HyperparameterTuner(tf_estimator,
                            objective_metric_name,
                            hyperparameter_ranges,
                            metric_definitions,
                            max_jobs=5,
                            max_parallel_jobs=2,
                            objective_type=objective_type)

In [ ]:
tuner.fit(inputs)

Go to Sagemaker console and under Training > Hyperparameter Tuning jobs > select the only entry. You will see different training jobs  with model configurations as well as a "Best Training job" tab.